### Installing and importing dependencies

In [1]:
!python --version

Python 3.10.12


In [8]:
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install matplotlib>=3.3.2

BRANCH = 'r1.23.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

"apt-get" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.21.1) to c:\users\777\appdata\local\temp\pip-install-udj_0dyb\nemo-toolkit_3e571863755a4b2a99d703e0ab11afd2
  Resolved https://github.com/NVIDIA/NeMo.git to commit 08af809442c9af250231380dda78505c4ce901c8
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
  Using cached numba-0.59.1-cp310-cp310-win_amd64.whl.metadata (2.8 kB)
  Using cached numpy-1.23.5-cp310-cp310-win_amd64.whl.metadata (2.3 kB)
  Using cached onnx-1.16.0-cp310-cp310-win_amd64.whl.metadata (16 kB)
  Using cached ruamel.yaml-0.18.6-py3-none-any.whl.metadata (23 kB)
  Using cached scikit_learn-1.4.2-cp310-cp310-win_

DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@r1.21.1#egg=nemo_toolkit[all] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git 'C:\Users\777\AppData\Local\Temp\pip-install-udj_0dyb\nemo-toolkit_3e571863755a4b2a99d703e0ab11afd2'
  Running command git checkout -b r1.21.1 --track origin/r1.21.1
  branch 'r1.21.1' set up to track 'origin/r1.21.1'.
  Switched to a new branch 'r1.21.1'
  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [53 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\pynini
  copying pynini\__in

In [9]:
from typing import List, Union

import torch
import torchaudio
import soundfile as sf
from omegaconf import DictConfig, ListConfig
import hydra

### Model for emotions classification

In [10]:
class SpecScaler(torch.nn.Module):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return torch.log(x.clamp_(1e-9, 1e9))


class GigaAMEmo(torch.nn.Module):
    def __init__(self, conf: Union[DictConfig, ListConfig]):
        super().__init__()
        self.id2name = conf.id2name
        self.feature_extractor = hydra.utils.instantiate(conf.feature_extractor)
        self.conformer = hydra.utils.instantiate(conf.encoder)
        self.linear_head = hydra.utils.instantiate(conf.classification_head)

    def forward(self, features, features_length=None):
        if features.dim() == 2:
            features = features.unsqueeze(0)
        if not features_length:
            features_length = torch.ones(features.shape[0]) * features.shape[-1]
            features_length = features_length.to(features.device)
        encoded, _ = self.conformer(audio_signal=features, length=features_length)
        encoded_pooled = torch.nn.functional.avg_pool1d(
            encoded, kernel_size=encoded.shape[-1]
        ).squeeze(-1)

        logits = self.linear_head(encoded_pooled)
        return logits

    def get_probs(self, audio_path: str) -> List[List[float]]:
        audio_signal, _ = sf.read(audio_path, dtype="float32")
        features = self.feature_extractor(torch.tensor(audio_signal).float().to(next(self.parameters()).device))
        logits = self.forward(features)
        probs = torch.nn.functional.softmax(logits).detach().tolist()
        return probs

### Downloading config, weights and audio example

In [11]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Loading weights, config and example wav for CTC-model
!wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/emo_model_weights.ckpt
!wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/emo_model_config.yaml
!wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/example.mp3

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


### Model instantiating and inference

In [12]:
%env HYDRA_FULL_ERROR=1
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/content/cache
%env HF_DATASETS_CACHE=/content/cache
%env CUDA_LAUNCH_BLOCKING=1

env: HYDRA_FULL_ERROR=1
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/content/cache
env: HF_DATASETS_CACHE=/content/cache
env: CUDA_LAUNCH_BLOCKING=1


In [13]:
from omegaconf import OmegaConf

model_config = 'emo_model_config.yaml'
model_weights = 'emo_model_weights.ckpt'
audio_path = 'angry.wav'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

conf = OmegaConf.load(model_config)
model = GigaAMEmo(conf)
ckpt = torch.load(model_weights, map_location="cpu")
model.load_state_dict(ckpt, strict=False)
model = model.to(device)
model.eval()
with torch.no_grad():
    probs = model.get_probs(audio_path)[0]
print(", ".join([f"{model.id2name[i]}: {p:.3f}" for i, p in enumerate(probs)]))

InstantiationException: Error locating target 'nemo.collections.asr.modules.ConformerEncoder', set env var HYDRA_FULL_ERROR=1 to see chained exception.
full_key: encoder

In [72]:
import pandas as pd
import os

In [92]:
df= pd.read_csv('colab_df_1.csv')

In [93]:
file_names = os.listdir('/content/sample_data/')

# Фильтруем датафрейм на основе списка имен файлов
filtered_df = df[df['audio_path'].apply(lambda x: os.path.basename(x) in file_names)]

In [94]:
filtered_df

,Unnamed: 0,audio_path,annotator_emo
0,37,30b1cf6769dbf1217c47c0d43584ec4d.wav,0.0
1,42,38bbd69cdc6618874a4768eae790204a.wav,0.0
2,49,60a09f6e5052f03d3c2f38f2f3e1fe8e.wav,0.0
5,73,0f312ab2416496272feb857bd2e7891b.wav,0.0
6,107,4a9a9092c8ff6fc26b9cd1418b049a56.wav,0.0
...,...,...,...
5002,113689,6cfceaa0b2819d918ca6a511fb6b80a8.wav,3.0
5003,113691,1b93a6a927f1524c3aed65f6b8d69973.wav,1.0
5004,113693,0f558fc6615cc0244df7d2f603bd08a2.wav,2.0
5006,113700,1fbf36955b644f738cbed9b9ea8c0186.wav,3.0


In [95]:
dic = {0:'neutral', 1:'sad', 2:'positive', 3:'angry'}
filtered_df['annotator_emo'] = filtered_df['annotator_emo'].map(dic)

[NeMo W 2024-04-24 15:45:40 nemo_logging:349] <ipython-input-95-3e72b67a3e65>:2: SettingWithCopyWarning: 
    A value is trying to be set on a copy of a slice from a DataFrame.
    Try using .loc[row_indexer,col_indexer] = value instead
    
    See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
      filtered_df['annotator_emo'] = filtered_df['annotator_emo'].map(dic)
    


In [96]:
filtered_df

,Unnamed: 0,audio_path,annotator_emo
0,37,30b1cf6769dbf1217c47c0d43584ec4d.wav,neutral
1,42,38bbd69cdc6618874a4768eae790204a.wav,neutral
2,49,60a09f6e5052f03d3c2f38f2f3e1fe8e.wav,neutral
5,73,0f312ab2416496272feb857bd2e7891b.wav,neutral
6,107,4a9a9092c8ff6fc26b9cd1418b049a56.wav,neutral
...,...,...,...
5002,113689,6cfceaa0b2819d918ca6a511fb6b80a8.wav,angry
5003,113691,1b93a6a927f1524c3aed65f6b8d69973.wav,sad
5004,113693,0f558fc6615cc0244df7d2f603bd08a2.wav,positive
5006,113700,1fbf36955b644f738cbed9b9ea8c0186.wav,angry


In [97]:
filtered_df['audio_path'] = filtered_df['audio_path'].apply(lambda x: os.path.join(r"/content/sample_data", x))

[NeMo W 2024-04-24 15:45:45 nemo_logging:349] <ipython-input-97-51be327f2e76>:1: SettingWithCopyWarning: 
    A value is trying to be set on a copy of a slice from a DataFrame.
    Try using .loc[row_indexer,col_indexer] = value instead
    
    See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
      filtered_df['audio_path'] = filtered_df['audio_path'].apply(lambda x: os.path.join(r"/content/sample_data", x))
    


In [98]:
filtered_df

,Unnamed: 0,audio_path,annotator_emo
0,37,/content/sample_data/30b1cf6769dbf1217c47c0d43...,neutral
1,42,/content/sample_data/38bbd69cdc6618874a4768eae...,neutral
2,49,/content/sample_data/60a09f6e5052f03d3c2f38f2f...,neutral
5,73,/content/sample_data/0f312ab2416496272feb857bd...,neutral
6,107,/content/sample_data/4a9a9092c8ff6fc26b9cd1418...,neutral
...,...,...,...
5002,113689,/content/sample_data/6cfceaa0b2819d918ca6a511f...,angry
5003,113691,/content/sample_data/1b93a6a927f1524c3aed65f6b...,sad
5004,113693,/content/sample_data/0f558fc6615cc0244df7d2f60...,positive
5006,113700,/content/sample_data/1fbf36955b644f738cbed9b9e...,angry


In [102]:
import torch
from sklearn.metrics import classification_report

# Предположим, что у вас уже есть загруженная модель model и датафрейм df

# Создаем список для хранения предсказанных меток и истинных меток
predicted_labels = []
true_labels = []

# Проходим по каждой строке в датафрейме и классифицируем файлы
for index, row in filtered_df.iterrows():
    file_path = row['audio_path']
    true_label = row['annotator_emo']

    # Получаем предсказанные вероятности для файла
    model.eval()
    with torch.no_grad():
        probs = model.get_probs(file_path)[0]
        probs = torch.tensor(probs)

    # Выбираем индекс класса с наивысшей вероятностью
    predicted_label = torch.argmax(probs).item()

    # Добавляем предсказанную и истинную метку в списки
    predicted_labels.append(predicted_label)
    true_labels.append(true_label)

# Создаем classification report



[NeMo W 2024-04-24 15:51:33 nemo_logging:349] <ipython-input-62-06a5deda234a>:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
      probs = torch.nn.functional.softmax(logits).detach().tolist()
    


In [105]:
dic = {'neutral':2, 'sad':1, 'positive':3, 'angry':0}
filtered_df['annotator_emo'] = filtered_df['annotator_emo'].map(dic)

[NeMo W 2024-04-24 15:58:12 nemo_logging:349] <ipython-input-105-02c14d653aa4>:2: SettingWithCopyWarning: 
    A value is trying to be set on a copy of a slice from a DataFrame.
    Try using .loc[row_indexer,col_indexer] = value instead
    
    See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
      filtered_df['annotator_emo'] = filtered_df['annotator_emo'].map(dic)
    


In [106]:
filtered_df

,Unnamed: 0,audio_path,annotator_emo
0,37,/content/sample_data/30b1cf6769dbf1217c47c0d43...,2
1,42,/content/sample_data/38bbd69cdc6618874a4768eae...,2
2,49,/content/sample_data/60a09f6e5052f03d3c2f38f2f...,2
5,73,/content/sample_data/0f312ab2416496272feb857bd...,2
6,107,/content/sample_data/4a9a9092c8ff6fc26b9cd1418...,2
...,...,...,...
5002,113689,/content/sample_data/6cfceaa0b2819d918ca6a511f...,0
5003,113691,/content/sample_data/1b93a6a927f1524c3aed65f6b...,1
5004,113693,/content/sample_data/0f558fc6615cc0244df7d2f60...,3
5006,113700,/content/sample_data/1fbf36955b644f738cbed9b9e...,0


In [107]:
true_labels = []
for index, row in filtered_df.iterrows():
    true_label = row['annotator_emo']
    true_labels.append(true_label)

In [109]:
report = classification_report(true_labels, predicted_labels)

In [111]:
print(report)

              precision    recall  f1-score   support

           0       0.88      0.86      0.87       413
           1       0.89      0.79      0.84       558
           2       0.71      0.80      0.75       593
           3       0.83      0.83      0.83       412

    accuracy                           0.82      1976
   macro avg       0.83      0.82      0.82      1976
weighted avg       0.82      0.82      0.82      1976

